In [1]:
!pip install --upgrade huggingface_hub transformers datasets faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 75.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 33.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 64.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 18.3 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.0
    Uninstalling fsspec-2025.3.0:
      Successfully uninstalled fsspec-2025.3.0
  Attempting uninstall: transformers
    Found existing installation: transformers 4.49.0
    Uninstalling transformers-4.49.0:
      Successfully uninstalled transformers-4.49.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are i

In [2]:
from google.colab import drive
from google.colab import userdata
import numpy as np
import faiss
from datasets import Dataset
from transformers import (
    DPRContextEncoder, DPRContextEncoderTokenizer,
    DPRQuestionEncoder, DPRQuestionEncoderTokenizer,
    AutoModelForCausalLM, AutoTokenizer
)
import torch
import re

In [3]:
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: write).
The token `notebooks` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `notebooks`


In [5]:
torch.set_grad_enabled(False)

ctx_encoder = DPRContextEncoder.from_pretrained("facebook/dpr-ctx_encoder-single-nq-base")
ctx_tokenizer = DPRContextEncoderTokenizer.from_pretrained("facebook/dpr-ctx_encoder-single-nq-base")

q_encoder = DPRQuestionEncoder.from_pretrained("facebook/dpr-question_encoder-single-nq-base")
q_tokenizer = DPRQuestionEncoderTokenizer.from_pretrained("facebook/dpr-question_encoder-single-nq-base")

config.json:   0%|          | 0.00/492 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

Some weights of the model checkpoint at facebook/dpr-ctx_encoder-single-nq-base were not used when initializing DPRContextEncoder: ['ctx_encoder.bert_model.pooler.dense.bias', 'ctx_encoder.bert_model.pooler.dense.weight']
- This IS expected if you are initializing DPRContextEncoder from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DPRContextEncoder from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

...

Some weights of the model checkpoint at facebook/dpr-question_encoder-single-nq-base were not used when initializing DPRQuestionEncoder: ['question_encoder.bert_model.pooler.dense.bias', 'question_encoder.bert_model.pooler.dense.weight']
- This IS expected if you are initializing DPRQuestionEncoder from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DPRQuestionEncoder from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [6]:
def read_file(file_path):
    with open(file_path, "r", encoding="utf-8") as f:
        return f.read()

def clean_text(text):
    paragraphs = text.split("\n\n")
    cleaned_paragraphs = [f"{i+1}. {p.strip()}" for i, p in enumerate(paragraphs) if p.strip()]
    return cleaned_paragraphs

def embed_text_batch(batch, max_length=512):
    texts = list(batch["text"])
    tokenized = ctx_tokenizer(
        texts, padding=True, truncation=True, max_length=max_length, return_tensors="pt"
    )
    with torch.no_grad():
        embeddings = ctx_encoder(**tokenized).pooler_output.numpy()
    return {"embeddings": embeddings.tolist()}


In [7]:
file_path = "pandp.txt"
text = read_file(file_path)
cleaned_paragraphs = clean_text(text)

In [8]:
cleaned_paragraphs

['1. *** START OF THE PROJECT GUTENBERG EBOOK 1342 ***\n                            [Illustration:',
 '2. GEORGE ALLEN\n                               PUBLISHER',
 '3. 156 CHARING CROSS ROAD\n                                LONDON',
 '4. RUSKIN HOUSE\n                                   ]',
 '5. [Illustration:',
 '6. _Reading Jane’s Letters._      _Chap 34._\n                                   ]',
 '8. PRIDE.\n                                  and\n                               PREJUDICE',
 '9. by\n                             Jane Austen,',
 '10. with a Preface by\n                           George Saintsbury\n                                  and\n                           Illustrations by\n                             Hugh Thomson',
 '11. [Illustration: 1894]',
 '12. Ruskin       156. Charing\n                       House.        Cross Road.',
 '13. London\n                             George Allen.',
 '15. CHISWICK PRESS:--CHARLES WHITTINGHAM AND CO.\n                  TOOKS COURT

In [9]:
batch_size = 20
ds = Dataset.from_dict({"text": cleaned_paragraphs, "paragraph_id": list(range(1, len(cleaned_paragraphs) + 1))})
ds = ds.map(embed_text_batch, batched=True, batch_size=batch_size).add_faiss_index(column="embeddings")

Map:   0%|          | 0/2510 [00:00<?, ? examples/s]

  0%|          | 0/3 [00:00<?, ?it/s]

In [10]:
def search(query, k=3):
    question_embedding = q_encoder(**q_tokenizer(query, return_tensors="pt"))[0][0].numpy()
    scores, retrieved_examples = ds.get_nearest_examples("embeddings", question_embedding, k=k)
    return [(retrieved_examples["paragraph_id"][i], retrieved_examples["text"][i]) for i in range(len(scores))]

In [11]:
# sample query
query = "Who is tolerable?"
results = search(query)

for para_id, text in results:
    print(f"Paragraph {para_id}: {text}\n")

Paragraph 2165: 2278. Darcy had walked away to another part of the room. She followed him with
her eyes, envied everyone to whom he spoke, had scarcely patience enough
to help anybody to coffee, and then was enraged against herself for
being so silly!

Paragraph 463: 484. “_I_ never saw such a woman. _I_ never saw such capacity, and taste, and
application, and elegance, as you describe, united.”

Paragraph 2051: 2160. “I should be sorry, indeed, if it were. _We_ were always good friends,
and now we are better.”



In [12]:
!pip install openai google-generativeai

In [15]:
# def compute_coverage(retrieved_paragraphs, ground_truth_paragraphs):
#     return len(set(retrieved_paragraphs) & set(ground_truth_paragraphs)) / len(ground_truth_paragraphs)

# def compute_citation(retrieved_paragraphs, ground_truth_paragraphs):
#     return len(set(retrieved_paragraphs) & set(ground_truth_paragraphs)) / len(retrieved_paragraphs)

In [17]:
# ground truth paras // TODO ADD MORE
ground_truth_paragraphs = {"Who is tolerable?": [242, 244]}

retrieved_paragraphs = [para_id for para_id, _ in results]
coverage_score = compute_coverage(retrieved_paragraphs, ground_truth_paragraphs[query])
citation_score = compute_citation(retrieved_paragraphs, ground_truth_paragraphs[query])

print(f"Coverage Score: {coverage_score:.2f}")
print(f"Citation Score: {citation_score:.2f}")


Coverage Score: 0.00
Citation Score: 0.00


In [31]:
from openai import OpenAI
from google import genai

google_client = genai.Client(api_key=userdata.get('GEMINI_API'))
openai_client = OpenAI(api_key=userdata.get('OPENAI_API_KEY'))

query = "Who is tolerable?"
def get_context(paragraphs):
    return "\n\n".join(paragraphs)

# truncate input context because token length
N = 300
truncated_full_context = get_context(cleaned_paragraphs[:N])

dpr_context = get_context([text for _, text in results])
oracle_context = get_context([cleaned_paragraphs[i-1] for i in ground_truth_paragraphs[query]])
full_context = get_context(cleaned_paragraphs)


def generate_gpt4o_answer(query, context):
    response = openai_client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "You are a helpful assistant. Use the context to answer the question and Always cite the paragraph number(s) like this: 36 or 175 in your answer."},
            {"role": "user", "content": f"Context:\n{context}\n\nQuestion: {query}"}
        ],
        temperature=0.3
    )
    return response.choices[0].message.content

def generate_gemini_answer(query, context):
    prompt = f"Context:\n{context}\n\nQuestion: {query}\nAnswer clearly and always include paragraph numbers (e.g. 36 or 157) if relevant."
    response = model.generate_content(prompt)
    response = google_client.models.generate_content(
        model="gemini-1.5-flash",
        contents=prompt,
    )
    return response.text

gpt4o_answers = {
    "dpr": generate_gpt4o_answer(query, dpr_context),
    "oracle": generate_gpt4o_answer(query, oracle_context),
    "full": generate_gpt4o_answer(query, truncated_full_context)
}

gemini_answers = {
    "dpr": generate_gemini_answer(query, dpr_context),
    "oracle": generate_gemini_answer(query, oracle_context),
    "full": generate_gemini_answer(query, truncated_full_context)
}



In [32]:
print("GPT-4o-mini FULL:\n", gpt4o_answers["full"])
print("GPT-4o-mini DPR:\n", gpt4o_answers["dpr"])
print("GPT-4o-mini Oracle:\n", gpt4o_answers["oracle"])

print("\nGemini-1.5-flash FULL:\n", gemini_answers["full"])
print("Gemini-1.5-flash DPR:\n", gemini_answers["dpr"])
print("Gemini-1.5-flash ORACLE:\n", gemini_answers["oracle"])

GPT-4o-mini FULL:
 Elizabeth Bennet is referred to as "tolerable" by Mr. Darcy during a conversation with Mr. Bingley at the assembly. Darcy coldly states, "She is tolerable: but not handsome enough to tempt me" (paragraph 246).
GPT-4o-mini DPR:
 The term "tolerable" is often associated with the character Mr. Darcy in Jane Austen's "Pride and Prejudice." In the context of the novel, Elizabeth Bennet refers to Mr. Darcy as "tolerable" when she first meets him, implying that while he is not particularly charming or agreeable to her, he is acceptable. However, the specific passages you provided do not explicitly mention who is considered "tolerable." If you are looking for a direct reference to this term, it is found in the earlier parts of the novel, particularly in Elizabeth's initial impressions of Darcy.
GPT-4o-mini Oracle:
 The context does not explicitly state who is considered "tolerable." However, it does describe a man who is not tolerable, as he is referred to as "a most disagre

In [29]:
# import re

# def extract_cited_paragraphs(answer_text):
#     matches = re.findall(r"(?:paragraph|para)?\s*\(?(\d{1,4})\)?", answer_text.lower())
#     return list(set(map(int, matches)))

# def evaluate_model_output(answer_text, ground_truth_ids):
#     cited = extract_cited_paragraphs(answer_text)
#     if not cited:
#         return 0.0, 0.0
#     coverage = compute_coverage(cited, ground_truth_ids)
#     citation = compute_citation(cited, ground_truth_ids)
#     return coverage, citation

# for model_name, outputs in [("GPT-4o", gpt4o_answers), ("Gemini", gemini_answers)]:
#     print(f"--- {model_name} ---")
#     for mode in ["dpr", "oracle", "full"]:
#         cov, cit = evaluate_model_output(outputs[mode], ground_truth_paragraphs[query])
#         print(f"{mode.upper()} → Coverage: {cov:.2f}, Citation: {cit:.2f}")


--- GPT-4o ---
DPR → Coverage: 0.00, Citation: 0.00
ORACLE → Coverage: 0.00, Citation: 0.00
FULL → Coverage: 0.00, Citation: 0.00
--- Gemini ---
DPR → Coverage: 0.00, Citation: 0.00
ORACLE → Coverage: 0.00, Citation: 0.00
FULL → Coverage: 0.00, Citation: 0.00
